In [ ]:
 import torch
import torch.nn as nn
from torch.utils.data import Dataset,DataLoader
import torchvision
import pandas as pd

In [ ]:
train_df=pd.read_csv('/content/drive/MyDrive/sign_mnist_train.csv')
valid_df=pd.read_csv('/content/drive/MyDrive/sign_mnist_valid.csv')

In [ ]:
device=torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [ ]:
class MyDataset(Dataset):
  def __init__(self,df):
    x_values=df.iloc[:,1:].values
    y_values=df.iloc[:,0].values
    x_values=x_values.reshape(-1,1,28,28)
    self.xs=torch.tensor(x_values,dtype=torch.float32).to(device)
    self.ys=torch.tensor(y_values,dtype=torch.int64).to(device)
  def __getitem__(self,index):
    return self.xs[index],self.ys[index]
  def __len__(self):
    return len(self.xs)

In [ ]:
training_data=MyDataset(train_df)
test_data=MyDataset(valid_df)

In [ ]:
train_loader=DataLoader(training_data,batch_size=64,shuffle=True)
test_loader=DataLoader(test_data,batch_size=64,shuffle=True)

In [ ]:
#2 Method
class first_cnn_block(nn.Module):
  def __init__(self,in_channels,out_channels,kernel_size,stride,padding):
    super(first_cnn_block,self).__init__()
    self.conv=nn.Conv2d(in_channels,out_channels,kernel_size,stride,padding)
    self.bn=nn.BatchNorm2d(out_channels)
    self.relu=nn.ReLU()
    self.maxpool=nn.MaxPool2d(kernel_size=2,stride=2)
  def forward(self,x):
    x=self.conv(x)
    x=self.bn(x)
    x=self.relu(x)
    x=self.maxpool(x)
    return x

In [ ]:
model1=nn.Sequential(
    first_cnn_block(1,25,3,1,1),
    first_cnn_block(25,50,3,1,1),
    first_cnn_block(50,75,3,1,1),
    nn.Flatten(),
    nn.Linear(75*3*3,512),
    nn.ReLU(),
    nn.Linear(512,24)
)

In [ ]:
model1=torch.compile(model1).to(device)

In [ ]:
loss_fn=nn.CrossEntropyLoss()
optimizer=torch.optim.Adam(model1.parameters(),lr=3e-4)

In [ ]:
def batch_accuracy(output,y,N):
  y_pred=torch.argmax(output,dim=1)
  correct=(y_pred==y).sum().item()
  return correct/N

In [ ]:
import torchvision.transforms.v2 as transforms
random_transforms=transforms.Compose(
    [transforms.RandomRotation(5),
     transforms.RandomHorizontalFlip(),
     transforms.ColorJitter(brightness=0.2,contrast=0.5)]
)

In [ ]:
def training():
  loss=0
  accuracy=0
  model1.train()
  for image,label in train_loader:
    image=image.to(device)
    label=label.to(device)
    output=model1(random_transforms(image))
    batch_loss=loss_fn(output,label)
    optimizer.zero_grad()
    batch_loss.backward()
    optimizer.step()
    loss+=batch_loss.item()
    accuracy+=batch_accuracy(output,label,label.shape[0])
  print(f'Training Accuracy: {accuracy/len(train_loader)}, Loss:{loss/len(train_loader)}')


In [ ]:
def validation():
  loss=0
  accuracy=0
  model1.eval()
  with torch.no_grad():
    for image,label in test_loader:
      image=image.to(device)
      label=label.to(device)
      output=model1(image)
      batch_loss=loss_fn(output,label)
      loss+=batch_loss.item()
      accuracy+=batch_accuracy(output,label,label.shape[0])
  print(f'Test Accuracy:{accuracy/len(test_loader)}, Loss: {loss/len(test_loader)}')


In [ ]:
epochs=10
for i in range(epochs):
  print(f'Epoch {i+1}')
  training()
  validation()

Epoch 1
Training Accuracy: 0.07273860398860399, Loss:3.1024348746924413
Test Accuracy:0.04798119469026549, Loss: 1391.2114895170769
Epoch 2
Training Accuracy: 0.08632108632108632, Loss:3.034402142593633
Test Accuracy:0.045077433628318585, Loss: 1193.123051196073
Epoch 3
Training Accuracy: 0.09309787434787435, Loss:3.0114169987765225
Test Accuracy:0.059734513274336286, Loss: 1205.6737190179065
Epoch 4
Training Accuracy: 0.09492243867243867, Loss:2.995461218284838
Test Accuracy:0.06222345132743363, Loss: 907.7430085038717
Epoch 5
Training Accuracy: 0.10657745032745032, Loss:2.9616263085034067
Test Accuracy:0.0517146017699115, Loss: 1269.2014149353568
Epoch 6
Training Accuracy: 0.10959700022200022, Loss:2.955699813949478
Test Accuracy:0.05448008849557522, Loss: 1271.4591572179204
Epoch 7
Training Accuracy: 0.12042008917008917, Loss:2.9166016498089946
Test Accuracy:0.05323561946902655, Loss: 1259.907401565957
Epoch 8
Training Accuracy: 0.11878746253746254, Loss:2.9227992298997645
Test Accu